In [1]:
import os
import shutil
import paddle
import paddle.dataset.flowers as flowers
import paddle.dataset.cifar as cifar
import paddle.fluid as fluid
paddle.enable_static()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


In [2]:
def resnet50(input):
    def conv_bn_layer(input,num_filters,filter_size,stride=1,act=None):
        conv=fluid.layers.conv2d(input=input,
                                num_filters=num_filters,
                                filter_size=filter_size,
                                stride=stride,
                                padding=(filter_size-1)//2,
                                act=None)#因为BN是在激活函数之前的，所以这里用None
        return fluid.layers.batch_norm(input=conv,act=act)
    def shortcut(input,ch_out,stride):
        ch_in=input.shape[1]#NCHW模式，即为通道数
        if ch_in!=ch_out or stride !=1:
            return conv_bn_layer(input,ch_out,1,stride)
        else:
            return input
    def bottleneck_block(input,num_filters,stride):
        conv0=conv_bn_layer(input=input,
                            num_filters=num_filters,
                            filter_size=1,
                            act='relu')
        conv1=conv_bn_layer(input=conv0,
                            num_filters=num_filters,
                            filter_size=3,
                            stride=stride,
                            act='relu')
        conv2=conv_bn_layer(input=conv1,
                            num_filters=num_filters*4,
                            filter_size=1,
                            act=None)
        short=shortcut(input,num_filters*4,stride)
        return fluid.layers.elementwise_add(x=short,y=conv2,act='relu')
    depth=[3,4,6,3]
    num_filters=[64,128,256,512]
    conv=conv_bn_layer(input=input, num_filters=64, filter_size=7, stride=2,act='relu')
    conv=fluid.layers.pool2d(input=conv,pool_size=(3,3),pool_stride=(2,2),pool_padding=1,pool_type='max')
    for block in range(len(depth)):
        for i in range(depth[block]):
            conv=bottleneck_block(input=conv,
                                num_filters=num_filters[block],
                                stride=2 if i==0 and block!=0 else 1)
    pool=fluid.layers.pool2d(input=conv,pool_size=7,pool_type='avg',global_pooling=True)
    return pool
Upsampling=paddle.nn.Upsample(size=[224,224],mode="bilinear")
image=fluid.data(name='image',shape=[None,3,32,32],dtype='float32')
temp=Upsampling(image)
pool=resnet50(temp)
model=fluid.layers.fc(input=pool,size=10,act='softmax')
label=fluid.data(name='label',shape=[None,1],dtype='int64')
cost=fluid.layers.cross_entropy(input=model, label=label)
avg_cost=fluid.layers.mean(cost)#返回值是个一维矩阵
acc=fluid.layers.accuracy(input=model, label=label)
test_program=fluid.default_main_program().clone(for_test=True)
optimizer=fluid.optimizer.AdamOptimizer(learning_rate=1e-3)
opts=optimizer.minimize(avg_cost)

In [3]:
train_reader=paddle.batch(cifar.train10(),batch_size=32)
test_reader=paddle.batch(cifar.test10(),batch_size=32)

In [4]:
place=fluid.CUDAPlace(0)
exe=fluid.Executor(place)
exe.run(fluid.default_startup_program())
feeder=fluid.DataFeeder(place=place,feed_list=[image,label])

In [7]:
for pass_id in range(10):
    for batch_id,data in enumerate(train_reader()):
        train_cost, train_acc=exe.run(program=fluid.default_main_program(),
                                    feed=feeder.feed(data),
                                    fetch_list=[avg_cost,acc])
        if batch_id %100==0:
            print("Pass:%d, Batch:%d, Cost:%0.5f, Accuracy:%0.5f" % (pass_id, batch_id,train_cost[0],train_acc[0]))
    test_accs=[]
    test_costs=[]
    for batch_id,data in enumerate(test_reader()):
        test_cost,test_acc=exe.run(program=test_program,
                                    feed=feeder.feed(data),
                                    fetch_list=[avg_cost,acc])
        test_accs.append(test_acc[0])
        test_costs.append(test_cost[0])
    test_cost=(sum(test_costs)/len(test_costs))
    test_acc=(sum(test_accs)/len(test_accs))
    print('Test:%d, Cost:%0.5f, Accuracy:%0.5f' % (pass_id, test_cost,test_acc))

Pass:0, Batch:0, Cost:3.49303, Accuracy:0.21875
Pass:0, Batch:100, Cost:2.11580, Accuracy:0.25000
Pass:0, Batch:200, Cost:2.02800, Accuracy:0.28125
Pass:0, Batch:300, Cost:1.74741, Accuracy:0.31250
Pass:0, Batch:400, Cost:1.89283, Accuracy:0.25000
Pass:0, Batch:500, Cost:1.72961, Accuracy:0.31250
Pass:0, Batch:600, Cost:1.51941, Accuracy:0.53125
Pass:0, Batch:700, Cost:1.52682, Accuracy:0.40625
Pass:0, Batch:800, Cost:1.60014, Accuracy:0.40625
Pass:0, Batch:900, Cost:1.24884, Accuracy:0.56250
Pass:0, Batch:1000, Cost:1.78913, Accuracy:0.25000
Pass:0, Batch:1100, Cost:1.53795, Accuracy:0.43750
Pass:0, Batch:1200, Cost:1.31215, Accuracy:0.50000
Pass:0, Batch:1300, Cost:1.22187, Accuracy:0.62500
Pass:0, Batch:1400, Cost:1.59256, Accuracy:0.56250
Pass:0, Batch:1500, Cost:1.21705, Accuracy:0.53125
Test:0, Cost:1.54362, Accuracy:0.47055
Pass:1, Batch:0, Cost:0.97998, Accuracy:0.62500
Pass:1, Batch:100, Cost:1.38248, Accuracy:0.56250
Pass:1, Batch:200, Cost:1.58800, Accuracy:0.34375
Pass:1, B

In [8]:
save_path='work/pretrained_model/'
shutil.rmtree(save_path, ignore_errors=True)
os.makedirs(save_path)
fluid.io.save_persistables(executor=exe,dirname=save_path)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 